Data-augmentation

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Apr  5 11:33:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    39W / 300W |   5957MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp /content/drive/MyDrive/Project/kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d ipythonx/wikiart-gangogh-creating-art-gan

100% 34.6G/34.6G [06:23<00:00, 114MB/s]
100% 34.6G/34.6G [06:24<00:00, 96.8MB/s]


In [ ]:
! unzip /content/wikiart-gangogh-creating-art-gan.zip -d /content/Training_Set

Streaming output truncated to the last 5000 lines.
  inflating: /content/Training_Set/still-life/55bcc9e1b914c4490f320b65efc91736c.jpg  
  inflating: /content/Training_Set/still-life/560140270766ce421fb8e6fdfc5d649ec.jpg  
  inflating: /content/Training_Set/still-life/562843932429acb8192900b1816d65bcc.jpg  
  inflating: /content/Training_Set/still-life/563e3706fd8258dfc750d91a11ef2cd8c.jpg  
  inflating: /content/Training_Set/still-life/56412c99702c33952475a32c962dcff0c.jpg  
  inflating: /content/Training_Set/still-life/5674ea7a997a3fae603e2cc3844aa808c.jpg  
  inflating: /content/Training_Set/still-life/5675cdb5e9488e298e1686bad3f60b46c.jpg  
  inflating: /content/Training_Set/still-life/567fcef3ee874b44e8bb5066839f581bc.jpg  
  inflating: /content/Training_Set/still-life/56826a3d23df262fca95687eaf755bbac.jpg  
  inflating: /content/Training_Set/still-life/568cc91147ad915386e1a9fcbf58d386c.jpg  
  inflating: /content/Training_Set/still-life/5698d9c5d5b1e40b97e12c46e7675fa7c.jpg  
  i

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
import torchvision.transforms as transform
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:
import os                
import pandas as pd
from torchvision.io import read_image

In [ ]:
class ODC(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 13]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Project')
import torch
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from utils import (
    gradient_penalty,
    plot_to_tensorboard,
    save_checkpoint,
    load_checkpoint,
    generate_examples,
)
from basepro import Discriminator, Generator
from math import log2
from tqdm import tqdm
import config
import random

torch.backends.cudnn.benchmarks = True


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
def get_loader(image_size):
    transform = transforms.Compose(
        [
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.Normalize(
                [0.5 for _ in range(config.CHANNELS_IMG)],
                [0.5 for _ in range(config.CHANNELS_IMG)],
            ),
        ]
    )
    batch_size = config.BATCH_SIZES[int(log2(image_size / 4))]
    #dataset = ODC(annotations_file = '/content/Training_Set/Training_Set/RFMiD_Training_Labels.csv',img_dir = '/content/Test_Set/Test_Set/Test', transform = transforms.ToTensor())
    dataset = datasets.ImageFolder(root='/content/Training_Set', transform=transform)
    #dataset = torch.utils.data.random_split(dataset, [1000,len(dataset)-1000])
    #n = random.randint(0,4000)
    #dataset = torch.utils.data.Subset(dataset, range(n,n+1000))
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=config.NUM_WORKERS,
        pin_memory=True,
    )
    return loader, dataset

In [ ]:
def train_fn(
    critic,
    gen,
    loader,
    dataset,
    step,
    alpha,
    opt_critic,
    opt_gen,
    tensorboard_step,
    writer,
    scaler_gen,
    scaler_critic,
):
    loop = tqdm(loader, leave=True)
    for batch_idx, (real, _) in enumerate(loop):
        real = real.to(config.DEVICE)
        cur_batch_size = real.shape[0]

        noise = torch.randn(cur_batch_size, config.Z_DIM, 1, 1).to(config.DEVICE)

        with torch.cuda.amp.autocast():
            fake = gen(noise, alpha, step)
            critic_real = critic(real, alpha, step)
            critic_fake = critic(fake.detach(), alpha, step)
            gp = gradient_penalty(critic, real, fake, alpha, step, device=config.DEVICE)
            loss_critic = (
                -(torch.mean(critic_real) - torch.mean(critic_fake))
                + config.LAMBDA_GP * gp
                + (0.001 * torch.mean(critic_real ** 2))
            )

        opt_critic.zero_grad()
        scaler_critic.scale(loss_critic).backward()
        scaler_critic.step(opt_critic)
        scaler_critic.update()

        with torch.cuda.amp.autocast():
            gen_fake = critic(fake, alpha, step)
            loss_gen = -torch.mean(gen_fake)

        opt_gen.zero_grad()
        scaler_gen.scale(loss_gen).backward()
        scaler_gen.step(opt_gen)
        scaler_gen.update()

        alpha += cur_batch_size / (
            (config.PROGRESSIVE_EPOCHS[step] * 0.5) * len(dataset)
        )
        alpha = min(alpha, 1)

        if batch_idx % 500 == 0:
            with torch.no_grad():
                fixed_fakes = gen(config.FIXED_NOISE, alpha, step) * 0.5 + 0.5
            plot_to_tensorboard(
                writer,
                loss_critic.item(),
                loss_gen.item(),
                real.detach(),
                fixed_fakes.detach(),
                tensorboard_step,
            )
            tensorboard_step += 1

        loop.set_postfix(
            gp=gp.item(),
            loss_critic=loss_critic.item(),
        )
    return tensorboard_step, alpha

In [ ]:
def main():
    gen = Generator(
        config.Z_DIM, config.IN_CHANNELS, img_channels=config.CHANNELS_IMG
    ).to(config.DEVICE)
    critic = Discriminator(config.IN_CHANNELS, img_channels=config.CHANNELS_IMG
    ).to(config.DEVICE)

    opt_gen = optim.Adam(gen.parameters(), lr=config.LEARNING_RATE, betas=(0.0, 0.99))
    opt_critic = optim.Adam(
        critic.parameters(), lr=config.LEARNING_RATE, betas=(0.0, 0.99)
    )
    scaler_critic = torch.cuda.amp.GradScaler()
    scaler_gen = torch.cuda.amp.GradScaler()

    writer = SummaryWriter(f"logs/gan1")

    if False:
        load_checkpoint(
            config.CHECKPOINT_GEN, gen, opt_gen, config.LEARNING_RATE,
        )
        load_checkpoint(
            config.CHECKPOINT_CRITIC, critic, opt_critic, config.LEARNING_RATE,
        )

    gen.train()
    critic.train()

    tensorboard_step = 0
    step = int(log2(config.START_TRAIN_AT_IMG_SIZE / 4))
    for num_epochs in config.PROGRESSIVE_EPOCHS[step:]:
        alpha = 1e-5
        loader, dataset = get_loader(4 * 2 ** step)
        print(f"Current image size: {4 * 2 ** step}")

        for epoch in range(num_epochs):
            print(f"Epoch [{epoch+1}/{num_epochs}]")
            tensorboard_step, alpha = train_fn(
                critic,
                gen,
                loader,
                dataset,
                step,
                alpha,
                opt_critic,
                opt_gen,
                tensorboard_step,
                writer,
                scaler_gen,
                scaler_critic,
            )

            if config.SAVE_MODEL:
                save_checkpoint(gen, opt_gen, filename=config.CHECKPOINT_GEN)
                save_checkpoint(critic, opt_critic, filename=config.CHECKPOINT_CRITIC)

        step += 1


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
if __name__ == "__main__":
    main()

Current image size: 64
Epoch [1/30]


 11%|█▏        | 171/1501 [01:54<14:31,  1.53it/s, gp=0.195, loss_critic=9.39]/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
 54%|█████▍    | 818/1501 [08:58<07:27,  1.53it/s, gp=0.0226, loss_critic=0.404]/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (99962094 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 60%|█████▉    | 895/1501 [09:49<06:36,  1.53it/s, gp=0.0226, loss_critic=-3.48]/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
 67%|██████▋   | 1006/1501 [11:02<05:26,  1.52it/s, gp=0.0452, loss_critic=-1.26]/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes b

=> Saving checkpoint
=> Saving checkpoint
Epoch [2/30]


 30%|███       | 451/1501 [04:56<11:28,  1.53it/s, gp=0.0779, loss_critic=-.484]/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (99962094 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 69%|██████▉   | 1041/1501 [11:23<05:01,  1.53it/s, gp=0.0522, loss_critic=-3.16]/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (107327830 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 69%|██████▉   | 1043/1501 [11:25<05:00,  1.53it/s, gp=0.0661, loss_critic=-1.96]/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 7. 
  warnings.warn(str(msg))
 75%|███████▌  | 1132/1501 [12:24<04:01,  1.53it/s, gp=0.106, loss_critic=-4.06]/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarnin

=> Saving checkpoint
=> Saving checkpoint
Epoch [3/30]


...

  6%|▌         | 91/1501 [02:53<44:12,  1.88s/it, gp=0.0258, loss_critic=-1.42]/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
 14%|█▎        | 205/1501 [06:27<40:39,  1.88s/it, gp=0.0294, loss_critic=-3.29]/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 7. 
  warnings.warn(str(msg))
 18%|█▊        | 265/1501 [08:20<38:42,  1.88s/it, gp=0.0305, loss_critic=-1.54]/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (107327830 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 78%|███████▊  | 1178/1501 [36:55<10:06,  1.88s/it, gp=0.0148, loss_critic=-1.98]/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (99962094 pixels) exceeds limi

=> Saving checkpoint
=> Saving checkpoint
Epoch [4/30]


  3%|▎         | 44/1501 [01:24<45:38,  1.88s/it, gp=0.0316, loss_critic=-1.62]/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 7. 
  warnings.warn(str(msg))
 43%|████▎     | 650/1501 [20:21<26:39,  1.88s/it, gp=0.065, loss_critic=-1.79]/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (107327830 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 45%|████▍     | 671/1501 [21:00<25:58,  1.88s/it, gp=0.058, loss_critic=-1.71]/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (99962094 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 78%|███████▊  | 1171/1501 [36:39<10:18,  1.87s/it, gp=0.0163, loss_critic=-2.67]/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: C

=> Saving checkpoint
=> Saving checkpoint
Epoch [5/30]


 28%|██▊       | 425/1501 [13:19<33:40,  1.88s/it, gp=0.0203, loss_critic=-1.24]/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (107327830 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
 33%|███▎      | 501/1501 [15:42<32:01,  1.92s/it, gp=0.0187, loss_critic=-1.87]/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
 55%|█████▌    | 833/1501 [26:05<20:53,  1.88s/it, gp=0.0255, loss_critic=-2.05]/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 7. 
  warnings.warn(str(msg))
 72%|███████▏  | 1080/1501 [33:49<13:09,  1.87s/it, gp=0.0164, loss_critic=-1.95]

In [ ]:
!pip install numba

In [ ]:
from numba import cuda
# all of your code and execution
cuda.select_device(0)
cuda.close()

In [ ]:
from numba import cuda
import torch 
device = cuda.get_current_device()
device.reset()
torch.cuda.is_available()

True

In [ ]:
CUDA_LAUNCH_BLOCKING=0